<a href="https://colab.research.google.com/github/LeticiaMoraesG/Projeto_gatos/blob/main/Projeto_semestral_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Template para o Colab do Projeto Semestral**
---

Atenção, podem ser que nem todas as tarefas sejam executadas no Colab (a aplicação por exemplo, pode estar hospedada no streamlit cloud). Mas a maior parte pode estar aqui ou ao menos indicada e comentada.


Além disso a entrega deve incluir:

1. **Um GitHub público do projeto**
2. **Código completo e executável em um notebook Python (este template)**
3. **Uma aplicação streamlit para consumo do modelo**
4. **Um texto/artigo do projeto**
5. **Um vídeo (link YouTube ou outro) de no máximo 3min de apresentação do projeto**

Um **`readme.md`** no GitHub público do projeto deve indicar (um índice) cada uma dessas entregas.








In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '10297494 Giovana Ribeiro de Franciso' #@param {type:"string"}
Aluno2 = '10402951, Leila Akina Ino' #@param {type:"string"}
Aluno3 = '10400969, Leticia Moraes Gutierrez de Oliveira' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}

In [ ]:
#@title Assinale aqui a sua opção de Projeto
Projeto = "IA Aplicada a Imagens: Uso de Modelos de Redes Neurais" #@param ["IA Aplicada a Imagens: Uso de Modelos de Redes Neurais", "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos"]

# **Resumo**

Apresente um "abstract" do seu projeto.

1. Objetivo do projeto
2. Fontes dos dados e dados originais (coletados)
3. Ferramentas/pacotes de IA a serem utilizados para a construção da solução
4. Um prévia dos resultados.

Este projeto tem como objetivo aplicar técnicas de Visão Computacional utilizando Deep Learning para classificar imagens de gatos em quatro raças distintas: Maine Coon, Sphynx, Persa e Siamês. A classificação é realizada por um modelo baseado na arquitetura EfficientNet, treinado com um conjunto de dados organizado em pastas separadas para cada raça, utilizando imagens reais de gatos.

As imagens foram coletadas e organizadas manualmente pelo grupo, e estão disponíveis em um repositório no GitHub.

O conjunto foi dividido em duas pastas principais:

treino/: usado para treinar o modelo

teste/: usado para avaliação do desempenho final
Cada uma contém subpastas com imagens das quatro raças.

Para o desenvolvimento da solução, foram utilizados os seguintes pacotes e frameworks:

- Python

- TensorFlow / Keras (para criação e treinamento do modelo)

- EfficientNet (com pesos pré-treinados no ImageNet)

- scikit-learn (para avaliação)

- Matplotlib / Seaborn (para visualização)

- Streamlit (para criar a aplicação web de classificação de imagens)

Como resultado parcial, o modelo não atinge uma acurácia muito boa, devido a poucas imagens no dataset durante a validação, no entando, é capaz de prever com boa confiabilidade a raça de gatos a partir de novas imagens. A aplicação final, desenvolvida com Streamlit, permite que o usuário envie uma imagem e receba a predição da raça com a respectiva confiança do modelo.

# **Apresentação dos dados**

[Github do Projeto](https://github.com/LeticiaMoraesG/Projeto_gatos/tree/main)

# Abaixo algumas seções de exemplo

> Pode haver mais, dependendo da sua aplicação. Para cada seção faça comentários explicando a tarefa e comentando/sumarizando os resultados.

In [1]:
!pip install tensorflow
!pip install pillow
!pip install scikit-learn
!pip install seaborn
!pip install matplotlib
!pip install pandas
!pip install numpy

In [2]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

# Imports completos - Execute esta célula DEPOIS da instalação
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import os
import time
from datetime import datetime
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import io

print("Todas as bibliotecas foram importadas com sucesso!")

TensorFlow version: 2.18.0
Todas as bibliotecas foram importadas com sucesso!


#efficientnet.py

##**Importação das Bibliotecas e Configurações Iniciais**
Importa todas as bibliotecas necessárias para o projeto de deep learning, visualização e avaliação do modelo e define todas as constantes e parâmetros que serão usados no treinamento, como tamanho das imagens, batch size, número de épocas, etc.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Configurações do modelo e treinamento
TAMANHO_IMG = (224, 224)  # Tamanho padrão para EfficientNet
TAMANHO_LOTE = 32         # Batch size para treinamento
EPOCAS = 10               # Número de épocas para fine-tuning
TAXA_APRENDIZADO = 0.001  # Learning rate inicial

# Definição das classes (raças de gatos)
RACAS = ['maine_coon', 'persa', 'siames', 'sphynx']
NUM_RACAS = len(RACAS)

# Caminhos dos diretórios de dados
PASTA_TREINO = 'treino'
PASTA_TESTE = 'teste'

## **Preparação e transformação dos dados**



###Configuração dos Geradores de Dados
Cria geradores de dados que aplicam transformações às imagens. Para treino, aplica data augmentation (rotação, zoom, etc.) para aumentar a diversidade dos dados. Para teste, apenas normaliza os pixels.

In [ ]:
# Gerador para dados de treino com data augmentation
# A augmentação ajuda a aumentar a diversidade dos dados e reduzir overfitting
gerador_treino = ImageDataGenerator(
    rescale=1./255,              # Normalização dos pixels [0,1]
    rotation_range=20,           # Rotação aleatória até 20 graus
    width_shift_range=0.2,       # Deslocamento horizontal até 20%
    height_shift_range=0.2,      # Deslocamento vertical até 20%
    horizontal_flip=True,        # Espelhamento horizontal
    zoom_range=0.2,              # Zoom aleatório até 20%
    shear_range=0.2,             # Cisalhamento até 20%
    fill_mode='nearest',         # Preenchimento de pixels vazios
    validation_split=0.2         # 20% dos dados para validação
)

# Gerador para teste (apenas normalização, sem augmentação)
gerador_teste = ImageDataGenerator(rescale=1./255)


###Carregamento dos Datasets
Carrega os dados de treino, validação e teste usando os geradores configurados. Divide automaticamente os dados de treino em 80% treino e 20% validação.

In [ ]:
# Carregar dados de treinamento (80% dos dados da pasta treino)
dados_treino = gerador_treino.flow_from_directory(
    PASTA_TREINO,
    target_size=TAMANHO_IMG,
    batch_size=TAMANHO_LOTE,
    class_mode='categorical',    # Para classificação multi-classe
    subset='training',
    shuffle=True                 # Embaralhar os dados
)

# Carregar dados de validação (20% dos dados da pasta treino)
dados_validacao = gerador_treino.flow_from_directory(
    PASTA_TREINO,
    target_size=TAMANHO_IMG,
    batch_size=TAMANHO_LOTE,
    class_mode='categorical',
    subset='validation',
    shuffle=False               # Não embaralhar para consistência
)

# Carregar dados de teste
dados_teste = gerador_teste.flow_from_directory(
    PASTA_TESTE,
    target_size=TAMANHO_IMG,
    batch_size=TAMANHO_LOTE,
    class_mode='categorical',
    shuffle=False
)


##**Construção do Modelo**

###Criação do Modelo
Define uma função que cria o modelo usando EfficientNetB0 como backbone (rede base) e adiciona camadas personalizadas para classificação das 4 raças de gatos e compila o modelo definindo o otimizador, função de perda e métricas que serão usadas durante o treinamento.

In [ ]:
def criar_modelo():
    """
    Cria o modelo usando EfficientNetB0 como backbone.

    Returns:
        modelo: Modelo completo para treinamento
        modelo_base: Modelo base EfficientNet para fine-tuning
    """
    # Carregar EfficientNetB0 pré-treinado no ImageNet
    modelo_base = EfficientNetB0(
        weights='imagenet',         # Pesos pré-treinados
        include_top=False,          # Remover camadas de classificação
        input_shape=(*TAMANHO_IMG, 3)
    )

    # Congelar as camadas base inicialmente (transfer learning)
    modelo_base.trainable = False

    # Construir o modelo completo
    entradas = tf.keras.Input(shape=(*TAMANHO_IMG, 3))
    x = modelo_base(entradas, training=False)  # Features do EfficientNet
    x = GlobalAveragePooling2D()(x)            # Pooling global
    x = Dropout(0.2)(x)                        # Regularização
    x = Dense(128, activation='relu')(x)       # Camada densa intermediária
    x = Dropout(0.2)(x)                        # Mais regularização
    saidas = Dense(NUM_RACAS, activation='softmax')(x)  # Classificação final

    modelo = Model(entradas, saidas)

    return modelo, modelo_base

# Criar o modelo
modelo, modelo_base = criar_modelo()

# Compilar o modelo
modelo.compile(
    optimizer=Adam(learning_rate=TAXA_APRENDIZADO),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Mostrar arquitetura do modelo
modelo.summary()

### Configuração de Callbacks
Define callbacks que monitoram o treinamento e fazem ajustes automáticos, como salvar o melhor modelo, parar o treinamento precocemente se não houver melhoria, e reduzir o learning rate.

In [ ]:
# Callback para salvar o melhor modelo
checkpoint = ModelCheckpoint(
    'melhor_modelo_raca_gatos.h5',
    monitor='val_accuracy',      # Monitorar acurácia de validação
    save_best_only=True,         # Salvar apenas o melhor modelo
    mode='max',                  # Maximizar a acurácia
    verbose=1
)

# Callback para parada precoce (evitar overfitting)
parada_precoce = EarlyStopping(
    monitor='val_loss',          # Monitorar perda de validação
    patience=5,                  # Parar se não melhorar por 5 épocas
    restore_best_weights=True,   # Restaurar melhores pesos
    verbose=1
)

# Callback para reduzir learning rate quando estagnado
reduzir_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,                  # Reduzir LR por fator de 0.2
    patience=3,                  # Aguardar 3 épocas sem melhoria
    min_lr=1e-7,                 # LR mínimo
    verbose=1
)

###**Treinamento de Camadas Superiores**
Treina apenas as camadas personalizadas (camadas superiores) enquanto mantém o backbone EfficientNet congelado. Isso é a primeira fase do transfer learning.

In [ ]:
print("=== FASE 1: TREINAMENTO DAS CAMADAS SUPERIORES ===")
print("Nesta fase, apenas as camadas personalizadas são treinadas.")
print("O backbone EfficientNet permanece congelado.\n")

# Treinar apenas as camadas superiores (transfer learning)
historico = modelo.fit(
    dados_treino,
    epochs=5,                    # Poucas épocas para esta fase
    validation_data=dados_validacao,
    callbacks=[checkpoint, parada_precoce, reduzir_lr],
    verbose=1
)

## **Fine Tuning do modelo**


###Preparação do Fine Tuning
Prepara o modelo para a segunda fase (fine-tuning) descongelando as últimas camadas do backbone EfficientNet e reduzindo o learning rate.

In [ ]:
print("\n=== FASE 2: FINE-TUNING ===")
print("Agora vamos descongelar parte do backbone para ajuste fino.")

# Descongelar o modelo base para fine-tuning
modelo_base.trainable = True

# Determinar quais camadas deixar treináveis (apenas as últimas 20)
ajuste_fino_em = len(modelo_base.layers) - 20

# Congelar as primeiras camadas, deixar apenas as últimas treináveis
for i, camada in enumerate(modelo_base.layers):
    if i < ajuste_fino_em:
        camada.trainable = False
    else:
        camada.trainable = True

print(f"- Total de camadas no backbone: {len(modelo_base.layers)}")
print(f"- Camadas congeladas: {ajuste_fino_em}")
print(f"- Camadas treináveis: {len(modelo_base.layers) - ajuste_fino_em}")

# Recompilar com learning rate menor para fine-tuning
modelo.compile(
    optimizer=Adam(learning_rate=TAXA_APRENDIZADO/10),  # LR 10x menor
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

##Execução
Executa a segunda fase do treinamento onde algumas camadas do backbone também são ajustadas com um learning rate menor para um ajuste mais refinado.

In [ ]:
# Continuar treinamento com fine-tuning
historico_ajuste = modelo.fit(
    dados_treino,
    epochs=EPOCAS,
    initial_epoch=len(historico.history['loss']),
    validation_data=dados_validacao,
    callbacks=[checkpoint, parada_precoce, reduzir_lr]
)


## **Avaliação do modelo**



##Avaliação do Conjunto de teste
Avalia o modelo treinado no conjunto de teste (dados nunca vistos durante o treinamento) para medir sua performance real.


In [ ]:
# Avaliar no conjunto de teste
print("\nAvaliando no conjunto de teste...")
perda_teste, acuracia_teste = modelo.evaluate(dados_teste)
print(f"Acurácia no teste: {acuracia_teste:.4f}")
print(f"Perda no teste: {perda_teste:.4f}")

##Visualização do Histórico de Treinamento
Cria gráficos mostrando a evolução da acurácia e perda durante o treinamento para visualizar como o modelo aprendeu e identificar possível overfitting.

In [ ]:
def plotar_historico_treinamento(historico, historico_ajuste=None):
    """
    Plota o histórico de treinamento mostrando acurácia e perda.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Combinar históricos se houver fine-tuning
    if historico_ajuste:
        acuracia = historico.history['accuracy'] + historico_ajuste.history['accuracy']
        acuracia_val = historico.history['val_accuracy'] + historico_ajuste.history['val_accuracy']
        perda = historico.history['loss'] + historico_ajuste.history['loss']
        perda_val = historico.history['val_loss'] + historico_ajuste.history['val_loss']

        # Marcar onde começou o fine-tuning
        fine_tuning_inicio = len(historico.history['loss'])
    else:
        acuracia = historico.history['accuracy']
        acuracia_val = historico.history['val_accuracy']
        perda = historico.history['loss']
        perda_val = historico.history['val_loss']
        fine_tuning_inicio = None

    # Plot acurácia
    ax1.plot(acuracia, label='Treino', linewidth=2)
    ax1.plot(acuracia_val, label='Validação', linewidth=2)
    ax1.set_title('Evolução da Acurácia', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('Acurácia')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    if fine_tuning_inicio:
        ax1.axvline(x=fine_tuning_inicio-0.5, color='red', linestyle='--',
                   label='Início Fine-tuning', alpha=0.7)
        ax1.legend()

    # Plot perda
    ax2.plot(perda, label='Treino', linewidth=2)
    ax2.plot(perda_val, label='Validação', linewidth=2)
    ax2.set_title('Evolução da Perda', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Época')
    ax2.set_ylabel('Perda')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    if fine_tuning_inicio:
        ax2.axvline(x=fine_tuning_inicio-0.5, color='red', linestyle='--',
                   label='Início Fine-tuning', alpha=0.7)
        ax2.legend()

    plt.tight_layout()
    plt.savefig('historico_treinamento.png', dpi=150, bbox_inches='tight')
    plt.show()

# Plotar histórico
plotar_historico_treinamento(historico, historico_ajuste)

##Matriz de Confusão e Análise Detalhada
Cria uma matriz de confusão para mostrar como o modelo confunde as diferentes raças e gera um relatório detalhado da performance por classe.

In [ ]:
dados_teste.reset()
predicoes = modelo.predict(dados_teste)
y_previsto = np.argmax(predicoes, axis=1)
y_verdadeiro = dados_teste.classes

# Criar matriz de confusão
matriz_confusao = confusion_matrix(y_verdadeiro, y_previsto)

# Plotar matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Blues',
            xticklabels=RACAS, yticklabels=RACAS)
plt.title('Matriz de Confusão')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.tight_layout()
plt.savefig('matriz_confusao.png')
plt.show()

## **Consumo do modelo**

###Função para Predição
Define uma função que permite usar o modelo treinado para classificar novas imagens de gatos, mostrando a raça prevista e a confiança da predição.

In [ ]:
# Função para fazer predições em novas imagens
def prever_raca_gato(caminho_imagem, modelo):
    # Carregar e preprocessar a imagem
    img = tf.keras.preprocessing.image.load_img(caminho_imagem, target_size=TAMANHO_IMG)
    array_img = tf.keras.preprocessing.image.img_to_array(img)
    array_img = np.expand_dims(array_img, axis=0)
    array_img = array_img / 255.0

    # Fazer predição
    predicoes = modelo.predict(array_img)
    indice_classe_prevista = np.argmax(predicoes[0])
    classe_prevista = RACAS[indice_classe_prevista]
    confianca = predicoes[0][indice_classe_prevista]

    # Mostrar resultados
    print(f"\nPredição para {caminho_imagem}:")
    print(f"Raça: {classe_prevista}")
    print(f"Confiança: {confianca:.2%}")
    print("\nProbabilidades para cada raça:")
    for i, raca in enumerate(RACAS):
        print(f"{raca}: {predicoes[0][i]:.2%}")

    return classe_prevista, confianca

###Relatório de Classificação e Salva o modelo final
Salva o modelo treinado em diferentes formatos para uso posterior, incluindo o modelo completo e apenas os pesos.

In [ ]:
# Relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_verdadeiro, y_previsto, target_names=RACAS))

# Salvar o modelo final
modelo.save('classificador_racas_gatos_final.h5')
print("\nModelo salvo como 'classificador_racas_gatos_final.h5'")

#app.py

##**Importação das Bibliotecas e Configurações Iniciais**
Importa todas as bibliotecas necessárias para o projeto de deep learning, visualização e avaliação do modelo e define todas as constantes.

In [ ]:
##import streamlit as st
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import os
import time
from datetime import datetime
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import io

## Configuração da Aplicação
Define título e layout wide do Streamlit, define os parâmetros do modelo e define as 4 raças de gatos que o modelo irá classificar.

In [ ]:
# Configuração da página
st.set_page_config(
    page_title="Classificador de Raças de Gatos",
    layout="wide"
)

# Título principal
st.title("Classificador de Raças de Gatos com EfficientNet")
st.markdown("---")

# Barra lateral
st.sidebar.title("Configurações")

# Configurações do modelo
TAMANHO_IMG = (224, 224)
TAMANHO_LOTE = st.sidebar.slider("Tamanho do Lote", 8, 64, 32)
EPOCAS = st.sidebar.slider("Épocas", 10, 50, 30)
TAXA_APRENDIZADO = st.sidebar.select_slider(
    "Taxa de Aprendizado",
    options=[0.0001, 0.0005, 0.001, 0.005, 0.01],
    value=0.001
)

RACAS = ['maine_coon', 'persa', 'siames', 'sphynx']
NUM_RACAS = len(RACAS)

EMOJIS_RACAS = {
    'maine_coon': '🦁',
    'persa': '😸',
    'siames': '😼',
    'sphynx': '🐈'
}

##Callback Personalizado para Streamlit
Classe herda de tf.keras.callbacks.Callback, mostra progresso do treinamento na interface web, armazena métricas de cada época para análise posterior e atualiza barras de progresso e métricas durante o treinamento

In [ ]:
# Callback personalizado para atualizar o Streamlit durante o treinamento
class StreamlitCallback(tf.keras.callbacks.Callback):
    def __init__(self, progress_bar, metrics_placeholder, epoch_placeholder, total_epochs):
        super().__init__()
        self.progress_bar = progress_bar
        self.metrics_placeholder = metrics_placeholder
        self.epoch_placeholder = epoch_placeholder
        self.total_epochs = total_epochs
        self.history_data = {
            'epoca': [],
            'perda': [],
            'acuracia': [],
            'perda_val': [],
            'acuracia_val': []
        }

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        # Atualizar progresso
        progress = (epoch + 1) / self.total_epochs
        self.progress_bar.progress(progress)

        # Atualizar época atual
        self.epoch_placeholder.markdown(f"### Época {epoch + 1}/{self.total_epochs}")

        # Guardar dados do histórico
        self.history_data['epoca'].append(epoch + 1)
        self.history_data['perda'].append(logs.get('loss', 0))
        self.history_data['acuracia'].append(logs.get('accuracy', 0))
        self.history_data['perda_val'].append(logs.get('val_loss', 0))
        self.history_data['acuracia_val'].append(logs.get('val_accuracy', 0))

        # Atualizar métricas na interface
        col1, col2, col3, col4 = self.metrics_placeholder.columns(4)
        col1.metric("Perda", f"{logs.get('loss', 0):.4f}")
        col2.metric("Acurácia", f"{logs.get('accuracy', 0):.4f}")
        col3.metric("Perda Val", f"{logs.get('val_loss', 0):.4f}")
        col4.metric("Acurácia Val", f"{logs.get('val_accuracy', 0):.4f}")

##Criação do Modelo
Criação do Modelo utilizando EfficientNetB0 pré-treinado no ImageNet; Congela pesos da base (trainable=False), @st.cache_resource evita recriar o modelo desnecessariamente

In [ ]:
@st.cache_resource
def criar_modelo():
    modelo_base = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(*TAMANHO_IMG, 3)
    )
    modelo_base.trainable = False

    entradas = tf.keras.Input(shape=(*TAMANHO_IMG, 3))
    x = modelo_base(entradas, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    saidas = Dense(NUM_RACAS, activation='softmax')(x)

    modelo = Model(entradas, saidas)
    return modelo, modelo_base

##Função de Avaliação
Gera probabilidades para cada classe, transforma probabilidades em classes preditas, como matriz de confusão e o relatório de classificação.

- Matriz de confusão: Mostra acertos/erros por classe;
- Relatório de classificação: Precisão, recall, F1-score por classe.

In [ ]:
def calcular_metricas_avaliacao(modelo, dados_teste):
    y_pred = modelo.predict(dados_teste, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = dados_teste.classes
    cm = confusion_matrix(y_true, y_pred_classes)
    report = classification_report(y_true, y_pred_classes,
                                 target_names=RACAS, output_dict=True)

    return cm, report, y_pred, y_true

##Dados

In [ ]:
# Abas principais
aba1, aba2, aba3 = st.tabs(["📊 Treinamento", "🔮 Predição", "📈 Análise"])

###Interface de treinamento de IA
Verifica diretórios de treino/teste e conta imagens por raça,exibe tabela e gráfico da distribuição do dataset, quando o botão é clicado, processa as imagens, cria o modelo de rede neural, treina com callbacks para monitoramento, mostra gráficos de acurácia/perda durante o treinamento e avalia o modelo no conjunto de teste e grava o modelo treinado em arquivo para uso posterior



In [ ]:
with aba1:
    st.header("Treinamento do Modelo")

    col1, col2 = st.columns(2)

    with col1:
        st.subheader("📁 Configuração dos Dados")
        diretorio_treino = st.text_input("Diretório de Treino", value="treino")
        diretorio_teste = st.text_input("Diretório de Teste", value="teste")

        # Verificar se os diretórios existem
        if os.path.exists(diretorio_treino) and os.path.exists(diretorio_teste):
            st.success("✅ Diretórios encontrados!")

            # Contar imagens
            contagem_treino = {}
            contagem_teste = {}

            for raca in RACAS:
                caminho_treino = os.path.join(diretorio_treino, raca)
                caminho_teste = os.path.join(diretorio_teste, raca)

                if os.path.exists(caminho_treino):
                    contagem_treino[raca] = len([f for f in os.listdir(caminho_treino)
                                               if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

                if os.path.exists(caminho_teste):
                    contagem_teste[raca] = len([f for f in os.listdir(caminho_teste)
                                              if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

            # Mostrar estatísticas
            st.subheader("📊 Estatísticas do Dataset")

            df_estatisticas = pd.DataFrame({
                'Raça': RACAS,
                'Emoji': [EMOJIS_RACAS[raca] for raca in RACAS],
                'Treino': [contagem_treino.get(raca, 0) for raca in RACAS],
                'Teste': [contagem_teste.get(raca, 0) for raca in RACAS]
            })

            st.dataframe(df_estatisticas, hide_index=True)

            # Gráfico de barras
            fig, ax = plt.subplots(figsize=(8, 4))
            x = np.arange(len(RACAS))
            largura = 0.35

            barras1 = ax.bar(x - largura/2, df_estatisticas['Treino'], largura, label='Treino')
            barras2 = ax.bar(x + largura/2, df_estatisticas['Teste'], largura, label='Teste')

            ax.set_xlabel('Raças')
            ax.set_ylabel('Número de Imagens')
            ax.set_title('Distribuição do Dataset')
            ax.set_xticks(x)
            ax.set_xticklabels(RACAS)
            ax.legend()

            st.pyplot(fig)

        else:
            st.error("❌ Diretórios não encontrados!")

    with col2:
        st.subheader("🚀 Iniciar Treinamento")

        if st.button("Treinar Modelo", type="primary"):
            if os.path.exists(diretorio_treino) and os.path.exists(diretorio_teste):

                try:
                    with st.spinner("Preparando dados..."):
                        # Criar geradores
                        gerador_treino = ImageDataGenerator(
                            rescale=1./255,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            horizontal_flip=True,
                            zoom_range=0.2,
                            shear_range=0.2,
                            fill_mode='nearest',
                            validation_split=0.2
                        )

                        gerador_teste = ImageDataGenerator(rescale=1./255)

                        dados_treino = gerador_treino.flow_from_directory(
                            diretorio_treino,
                            target_size=TAMANHO_IMG,
                            batch_size=TAMANHO_LOTE,
                            class_mode='categorical',
                            subset='training'
                        )

                        dados_validacao = gerador_treino.flow_from_directory(
                            diretorio_treino,
                            target_size=TAMANHO_IMG,
                            batch_size=TAMANHO_LOTE,
                            class_mode='categorical',
                            subset='validation'
                        )

                        dados_teste = gerador_teste.flow_from_directory(
                            diretorio_teste,
                            target_size=TAMANHO_IMG,
                            batch_size=TAMANHO_LOTE,
                            class_mode='categorical',
                            shuffle=False
                        )

                    # Criar modelo
                    with st.spinner("Criando modelo..."):
                        modelo, modelo_base = criar_modelo()
                        modelo.compile(
                            optimizer=Adam(learning_rate=TAXA_APRENDIZADO),
                            loss='categorical_crossentropy',
                            metrics=['accuracy']
                        )

                    # Callbacks
                    checkpoint = ModelCheckpoint(
                        'melhor_modelo_racas_gatos.h5',
                        monitor='val_accuracy',
                        save_best_only=True,
                        mode='max',
                        verbose=1
                    )

                    parada_precoce = EarlyStopping(
                        monitor='val_loss',
                        patience=5,
                        restore_best_weights=True,
                        verbose=1
                    )

                    # Placeholders para métricas
                    espaco_epoca = st.empty()
                    espaco_metricas = st.empty()
                    barra_progresso = st.progress(0)

                    # Callback personalizado para Streamlit
                    callback_streamlit = StreamlitCallback(
                        barra_progresso, espaco_metricas, espaco_epoca, EPOCAS
                    )

                    st.info("🏃 Treinamento em andamento...")

                    historico = modelo.fit(
                        dados_treino,
                        epochs=EPOCAS,
                        validation_data=dados_validacao,
                        callbacks=[checkpoint, parada_precoce, callback_streamlit],
                        verbose=0
                    )

                    st.success("✅ Treinamento concluído!")

                    # Salvar histórico no session state
                    st.session_state['historico_treinamento'] = callback_streamlit.history_data
                    st.session_state['modelo_treinado'] = True
                    st.session_state['modelo'] = modelo
                    st.session_state['dados_teste'] = dados_teste

                    # Plotar gráficos do histórico real
                    st.subheader("📈 Histórico de Treinamento")

                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

                    # Acurácia
                    ax1.plot(callback_streamlit.history_data['epoca'],
                            callback_streamlit.history_data['acuracia'], label='Treino')
                    ax1.plot(callback_streamlit.history_data['epoca'],
                            callback_streamlit.history_data['acuracia_val'], label='Validação')
                    ax1.set_xlabel('Época')
                    ax1.set_ylabel('Acurácia')
                    ax1.set_title('Acurácia por Época')
                    ax1.legend()
                    ax1.grid(True)

                    # Perda
                    ax2.plot(callback_streamlit.history_data['epoca'],
                            callback_streamlit.history_data['perda'], label='Treino')
                    ax2.plot(callback_streamlit.history_data['epoca'],
                            callback_streamlit.history_data['perda_val'], label='Validação')
                    ax2.set_xlabel('Época')
                    ax2.set_ylabel('Perda')
                    ax2.set_title('Perda por Época')
                    ax2.legend()
                    ax2.grid(True)

                    st.pyplot(fig)

                    # Salvar modelo
                    modelo.save('classificador_racas_gatos_streamlit.h5')

                    # Avaliar no conjunto de teste
                    with st.spinner("Avaliando modelo no conjunto de teste..."):
                        avaliacao = modelo.evaluate(dados_teste, verbose=0)
                        st.session_state['avaliacao_teste'] = {
                            'perda': avaliacao[0],
                            'acuracia': avaliacao[1]
                        }

                        # Calcular métricas detalhadas
                        cm, report, y_pred, y_true = calcular_metricas_avaliacao(modelo, dados_teste)
                        st.session_state['matriz_confusao'] = cm
                        st.session_state['relatorio_classificacao'] = report

                    st.success(f"🎯 Acurácia no teste: {avaliacao[1]:.1%}")

                except Exception as e:
                    st.error(f"❌ Erro durante o treinamento: {str(e)}")
                    st.exception(e)

            else:
                st.error("Por favor, verifique os diretórios de dados!")

###Identificador de raças de gatos
Verifica se existe um modelo treinado na sessão ou salvo em arquivo, permite upload de imagem de gato (JPG, JPEG, PNG), redimensiona e ajusta a foto para o formato que o modelo entende, usa o modelo de IA para identificar a raça do gato, exibe a raça identificada, confiança e gráfico com todas as probabilidades e apresenta curiosidades sobre a raça identificada

In [ ]:
with aba2:
    st.header("Predição de Raça")

    # Verificar se existe modelo treinado
    caminho_modelo = 'classificador_racas_gatos_streamlit.h5'

    modelo = None
    if 'modelo' in st.session_state:
        modelo = st.session_state['modelo']
        st.success("✅ Modelo da sessão carregado!")
    elif os.path.exists(caminho_modelo):
        # Carregar modelo
        @st.cache_resource
        def carregar_modelo():
            return tf.keras.models.load_model(caminho_modelo)

        modelo = carregar_modelo()
        st.success("✅ Modelo salvo carregado!")

    if modelo is not None:
        # Upload de imagem
        arquivo_enviado = st.file_uploader(
            "Escolha uma imagem de gato",
            type=['jpg', 'jpeg', 'png'],
            help="Formatos suportados: JPG, JPEG, PNG"
        )

        if arquivo_enviado is not None:
            # Mostrar imagem
            col1, col2 = st.columns(2)

            with col1:
                st.subheader("📷 Imagem Original")
                imagem = Image.open(arquivo_enviado)
                st.image(imagem, use_column_width=True)
                st.caption(f"Dimensões: {imagem.size[0]} x {imagem.size[1]}")
                st.caption(f"Formato: {imagem.format}")

            with col2:
                st.subheader("🔮 Predição")

                # Preprocessar imagem
                img = imagem.resize(TAMANHO_IMG)
                array_img = np.array(img)

                # Verificar se a imagem tem 3 canais (RGB)
                if len(array_img.shape) == 2:
                    array_img = np.stack([array_img] * 3, axis=-1)
                elif array_img.shape[2] == 4:
                    array_img = array_img[:, :, :3]

                array_img = np.expand_dims(array_img, axis=0)
                array_img = array_img / 255.0

                with st.spinner("Analisando..."):
                    predicoes = modelo.predict(array_img, verbose=0)
                    indice_predito = np.argmax(predicoes[0])
                    raca_predita = RACAS[indice_predito]
                    confianca = predicoes[0][indice_predito]

                # Mostrar resultado principal
                st.markdown(f"### {EMOJIS_RACAS[raca_predita]} {raca_predita.replace('_', ' ').title()}")
                st.markdown(f"**Confiança:** {confianca:.1%}")

                # Barra de progresso para confiança
                st.progress(float(confianca))

                # Mostrar todas as probabilidades
                st.markdown("#### Probabilidades por Raça:")

                # Criar DataFrame com resultados
                df_resultados = pd.DataFrame({
                    'Raça': [f"{EMOJIS_RACAS[raca]} {raca.replace('_', ' ').title()}" for raca in RACAS],
                    'Probabilidade': [f"{predicoes[0][i]:.1%}" for i in range(NUM_RACAS)],
                    'Score': predicoes[0]
                })
                df_resultados = df_resultados.sort_values('Score', ascending=False)

                # Mostrar como gráfico de barras
                fig, ax = plt.subplots(figsize=(8, 4))
                barras = ax.barh(df_resultados['Raça'], df_resultados['Score'])

                # Colorir a barra com maior probabilidade
                cores = ['#1f77b4' if i != 0 else '#ff7f0e' for i in range(len(barras))]
                for barra, cor in zip(barras, cores):
                    barra.set_color(cor)

                ax.set_xlabel('Probabilidade')
                ax.set_xlim(0, 1)
                ax.set_title('Distribuição de Probabilidades')

                # Adicionar valores nas barras
                for i, (raca, score) in enumerate(zip(df_resultados['Raça'], df_resultados['Score'])):
                    ax.text(score + 0.01, i, f'{score:.1%}', va='center')

                st.pyplot(fig)

                # Informações adicionais sobre a raça
                info_racas = {
                    'maine_coon': "O Maine Coon é uma das maiores raças de gatos domésticos. São conhecidos por serem gentis gigantes, muito sociáveis e brincalhões.",
                    'persa': "O gato Persa é conhecido por sua pelagem longa e luxuosa e face achatada. São gatos calmos, gentis e afetuosos.",
                    'siames': "O gato Siamês é uma raça elegante conhecida por seus olhos azuis e pelagem clara com extremidades escuras. São muito vocais e inteligentes.",
                    'sphynx': "O Sphynx é conhecido por ser 'sem pelos' (na verdade tem uma fina camada de penugem). São gatos muito afetuosos e energéticos."
                }

                with st.expander(f"ℹ️ Sobre {raca_predita.replace('_', ' ').title()}"):
                    st.write(info_racas.get(raca_predita, "Informação não disponível."))

    else:
        st.warning("⚠️ Nenhum modelo treinado encontrado. Por favor, treine um modelo primeiro na aba 'Treinamento'.")

###Análise do Modelo
Exibe acurácia, perda, precisão e recall do modelo em cartões visuais,cria um mapa de calor mostrando onde o modelo acerta e erra cada raça, apresenta tabela com precisão, recall e F1-score de cada raça individualmente,compara visualmente as métricas de todas as raças em formato circular, exibe gráficos da evolução do treinamento e detecta possível overfitting e mostra se o modelo decorou os dados de treino ao invés de aprender

In [ ]:
with aba3:
    st.header("Análise do Modelo")

    if st.session_state.get('modelo_treinado', False) and 'avaliacao_teste' in st.session_state:

        # Métricas
        st.subheader("📊 Métricas de Performance")

        avaliacao = st.session_state['avaliacao_teste']
        relatorio = st.session_state.get('relatorio_classificacao', {})

        col1, col2, col3, col4 = st.columns(4)

        # Valores do modelo
        acuracia_final = avaliacao['acuracia']
        perda_final = avaliacao['perda']

        # Calcular precisão e recall médios
        if relatorio and 'macro avg' in relatorio:
            precisao_media = relatorio['macro avg']['precision']
            recall_medio = relatorio['macro avg']['recall']
        else:
            precisao_media = acuracia_final
            recall_medio = acuracia_final

        col1.metric("Acurácia Final", f"{acuracia_final:.1%}")
        col2.metric("Perda Final", f"{perda_final:.3f}")
        col3.metric("Precisão Média", f"{precisao_media:.1%}")
        col4.metric("Recall Médio", f"{recall_medio:.1%}")

        # Matriz de Confusão
        if 'matriz_confusao' in st.session_state:
            st.subheader("🎯 Matriz de Confusão")

            matriz_confusao = st.session_state['matriz_confusao']

            fig, ax = plt.subplots(figsize=(8, 6))
            sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Blues',
                        xticklabels=RACAS, yticklabels=RACAS, ax=ax)
            ax.set_title('Matriz de Confusão')
            ax.set_xlabel('Predito')
            ax.set_ylabel('Verdadeiro')
            st.pyplot(fig)

        # Métricas por classe
        if relatorio:
            st.subheader("📈 Performance por Raça")

            dados_metricas = {
                'Raça': [f"{EMOJIS_RACAS[raca]} {raca.replace('_', ' ').title()}" for raca in RACAS],
                'Precisão': [relatorio.get(raca, {}).get('precision', 0) for raca in RACAS],
                'Recall': [relatorio.get(raca, {}).get('recall', 0) for raca in RACAS],
                'F1-Score': [relatorio.get(raca, {}).get('f1-score', 0) for raca in RACAS]
            }

            df_metricas = pd.DataFrame(dados_metricas)

            # Mostrar tabela
            st.dataframe(
                df_metricas.style.format({
                    'Precisão': '{:.1%}',
                    'Recall': '{:.1%}',
                    'F1-Score': '{:.1%}'
                }),
                hide_index=True
            )

            # Gráfico radar com dados
            st.subheader("🕸️ Comparação de Métricas")

            categorias = ['Precisão', 'Recall', 'F1-Score']

            fig, ax = plt.subplots(figsize=(8, 6), subplot_kw=dict(projection='polar'))

            angulos = np.linspace(0, 2 * np.pi, len(categorias), endpoint=False).tolist()
            angulos += angulos[:1]

            for idx, raca in enumerate(RACAS):
                valores = [dados_metricas['Precisão'][idx],
                          dados_metricas['Recall'][idx],
                          dados_metricas['F1-Score'][idx]]
                valores += valores[:1]

                ax.plot(angulos, valores, 'o-', linewidth=2,
                       label=f"{EMOJIS_RACAS[raca]} {raca.replace('_', ' ').title()}")
                ax.fill(angulos, valores, alpha=0.15)

            ax.set_theta_offset(np.pi / 2)
            ax.set_theta_direction(-1)
            ax.set_xticks(angulos[:-1])
            ax.set_xticklabels(categorias)
            ax.set_ylim(0, 1)
            ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
            ax.set_yticklabels(['20%', '40%', '60%', '80%', '100%'])
            ax.grid(True)
            ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

            st.pyplot(fig)

        # Mostrar histórico de treinamento
        if 'historico_treinamento' in st.session_state:
            st.subheader("📈 Histórico de Treinamento Detalhado")

            historico = st.session_state['historico_treinamento']

            # Criar gráficos do histórico
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))

            # Acurácia
            ax1.plot(historico['epoca'], historico['acuracia'], label='Treino', marker='o')
            ax1.plot(historico['epoca'], historico['acuracia_val'], label='Validação', marker='s')
            ax1.set_xlabel('Época')
            ax1.set_ylabel('Acurácia')
            ax1.set_title('Acurácia por Época')
            ax1.legend()
            ax1.grid(True)

            # Perda
            ax2.plot(historico['epoca'], historico['perda'], label='Treino', marker='o')
            ax2.plot(historico['epoca'], historico['perda_val'], label='Validação', marker='s')
            ax2.set_xlabel('Época')
            ax2.set_ylabel('Perda')
            ax2.set_title('Perda por Época')
            ax2.legend()
            ax2.grid(True)

            # Diferença entre treino e validação (Overfitting)
            diff_acc = np.array(historico['acuracia']) - np.array(historico['acuracia_val'])
            ax3.plot(historico['epoca'], diff_acc, marker='o', color='red')
            ax3.set_xlabel('Época')
            ax3.set_ylabel('Diferença de Acurácia')
            ax3.set_title('Overfitting (Treino - Validação)')
            ax3.grid(True)
            ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)

            ax4.bar(range(len(RACAS)), [1]*len(RACAS), color=['blue', 'green', 'red', 'orange'])
            ax4.set_xlabel('Raças')
            ax4.set_ylabel('Normalizado')
            ax4.set_title('Distribuição de Classes')
            ax4.set_xticks(range(len(RACAS)))
            ax4.set_xticklabels(RACAS, rotation=45)

            plt.tight_layout()
            st.pyplot(fig)

    else:
        st.warning("⚠️ Nenhum modelo treinado encontrado. Por favor, treine um modelo primeiro.")

# **Referências**

Este é um item obrigatório. Inclua aqui o as referências, fontes, ou bibliografia e sites/bibliotecas que foram empregados para construir a sua proposta.

Gutierrez, L. M. Projeto Gatos. 2024. Repositório GitHub. Disponível em: https://github.com/LeticiaMoraesG/Projeto_gatos/tree/main. Acesso em: 20 mai. 2025.
KERAS. EfficientNet. In: Keras Applications Documentation. Disponível em: https://keras.io/api/applications/efficientnet/. Acesso em: 27 mai. 2025.
STREAMLIT. Streamlit: The fastest way to build and share data apps. Disponível em: https://streamlit.io/. Acesso em: 27 mai. 2025.
TAN, M. et al. EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks. arXiv preprint arXiv:1905.11946, 2019. Disponível em: https://arxiv.org/abs/1905.11946. Acesso em: 28 mai. 2025.
TENSORFLOW. TensorFlow: An end-to-end open source machine learning platform. Disponível em: https://www.tensorflow.org/. Acesso em: 27 mai. 2025.

---

In [ ]:
#@title **Avaliação**
GitHub = 10 #@param {type:"slider", min:0, max:10, step:1}

Implementacao_Model_Code = 7 #@param {type:"slider", min:0, max:10, step:1}

Aplicacao_Streamlit = 9 #@param {type:"slider", min:0, max:10, step:1}

Texto_Artigo  = 6 #@param {type:"slider", min:0, max:10, step:1}

Video = 7 #@param {type:"slider", min:0, max:10, step:1}

Geral = 7 #@param {type:"slider", min:0, max:10, step:1}

In [ ]:
#@title **Nota Final**

nota = 2*GitHub + 4*Implementacao_Model_Code + 2*Aplicacao_Streamlit + 1*Texto_Artigo + 1*Video

nota = nota / 10

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 7.9



,tia,nome,nota
0,1115665,ADRIANA FUJITA,7.9
1,1115677,DANIEL HENRIQUE,7.9
